In [ ]:
%%capture
# Для Collab
# !pip install langchain==1.1.3 langchain-mistralai==1.1.0 langchain-text-splitters==1.0.0 faiss-cpu==1.13.1 mistralai==1.9.11 langchain-community==0.4.1

In [ ]:
%%capture
# Чтобы скачать трейн датасет документов (если не работает, есть ссылка в README)
!wget https://huggingface.co/datasets/irtez/ITMO-LLM-RAG-test/resolve/main/questions_data.zip?download=true -O questions_data.zip
!unzip questions_data.zip

In [ ]:
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from typing import List
import json
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from pathlib import Path
from tqdm.notebook import tqdm

In [ ]:
# Положите ключ в .env файл в директории с ноутбуком или введите его вручную
from dotenv import load_dotenv
load_dotenv()
MISTRAL_API_KEY = ''
assert MISTRAL_API_KEY or os.getenv("MISTRAL_API_KEY"), "Введите ключ"

In [ ]:
# Можно использовать любую модель от любого провайдера, Mistral тут для примера
chat = ChatMistralAI(
    api_key=os.getenv("MISTRAL_API_KEY") or MISTRAL_API_KEY,
    model_name='mistral-large-2407'
)

# Загрузка вопросов

In [ ]:
# Считываем вопросы
questions = []
with open('questions.jsonl', 'r') as f:
    for line in f:
        questions.append(json.loads(line))

In [ ]:
# Считываем метадату документов (в основном, время редактирования - то есть на какой момент документ актуален)
# Пока что нигде не используется, но в датасете есть вопросы, связанные со временем
with open('docs_metadata.json', 'r') as f:
    docs_metadata = json.load(f)
docs_metadata['7.html']

In [ ]:
# Не заработает для валидационного датасета
# {q['question_type'] for q in questions}

Типы вопросов (`question_type`):
- Simple - простой вопрос, например, дата рождения или авторы книги
- Simple with condition - простые вопросы с условиями, например, цена акции в определенную дату
- Set - ответ на вопрос - это список сущностей (*Какие на земле есть океаны?*)
- Multi-hop - вопросы, для ответа на которые нужно сделать несколько "шагов" поиска информации, например: *Сколько турниров по всему миру выиграл рекордсмен чемпионата Argentine PGA?* (нужно сначала найти, кто является рекордсменом, а потом - сколько турниров он выиграл, и только затем дать ответ)
- False premise - Вопрос поставлен некорректно, верные ответы - "Я не знаю", "Я не могу ответить", "Вопрос составлен некорректно"
- Aggregation - для ответа на вопрос нужна аггрегация разных ответов
- Comparison - для ответа на вопрос нужно сравнить сущности между собой (*Кто начал выступать раньше, Adele или Ed Sheeran?*)

На любые вопросы ответа может не быть (правильный ответ LLM - "Не знаю" или "Не могу ответить из контекста").

**ВАЖНО:** типы вопросов, ответы на эти вопросы, а также список документов, релевантных для вопроса (поле `documents`) не будут доступны на валидационном датасете, который будет выдан на паре. 

# Загрузка и чанкинг документов

In [ ]:
def load_all_documents(docs_dir: str = "questions_data") -> list[Document]:
    """Загружает все HTML документы из папки."""
    docs_path = Path(docs_dir)
    documents = []
    
    for file in tqdm(sorted(docs_path.glob("*")), desc="Loading documents"):
        with open(file, 'r', encoding='utf-8') as f:
            content = f.read()
        file_name = file.name
        # Создаем Document с метаданными
        doc_metadata = docs_metadata[file_name]
        doc_metadata["source"] = file_name
        doc = Document(
            page_content=content,
            metadata=doc_metadata
        )
        documents.append(doc)
    
    print(f"Загружено документов: {len(documents)}")
    return documents

# Загружаем все документы
all_docs = load_all_documents(docs_dir="questions_data")

In [ ]:
# Инициализируем эмбеддинги. Будем использовать all-MiniLM-L6-v2
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cuda'} # or CPU
)

# Простой сплиттер
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

In [ ]:
# Разбиваем документы на чанки
all_chunks = text_splitter.split_documents(all_docs)
print(f"Всего чанков: {len(all_chunks)}")

# Подсказка: в датасете много HTML документов. Чтобы уменьшить количество чанков, можно произвести их предобработку (очистку от мусора)

In [ ]:
# Создаем FAISS индекс
vectorstore = FAISS.from_documents(all_chunks, embeddings)

# Сам RAG (бейзлайн)

In [ ]:
# Промпт для LLM

RAG_SYSTEM_PROMPT = """You are a precise question-answering assistant. Your task is to answer questions based ONLY on the provided context documents.

CRITICAL RULES:
1. ONLY use information explicitly stated in the context below
2. If the context doesn't contain enough information to answer, respond with "I cannot answer this question based on the provided information"
3. Do NOT use any prior knowledge - ONLY the context
4. Be concise and direct in your answers

CONTEXT:
{context}"""

RAG_USER_PROMPT = """Question: {question}
Question time: {question_time}

First, identify if the question can be answered from the context above.
Then provide your answer."""

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", RAG_SYSTEM_PROMPT),
    ("human", RAG_USER_PROMPT)
])

# Создаем простую цепочку
rag_chain = rag_prompt | chat | StrOutputParser()

In [ ]:
def print_chunks(retrieved_docs: List[Document]) -> None:
    for doc in retrieved_docs:
        print(f"Документ '{doc.metadata['source']}'")
        print('Содержимое чанка:\n')
        print(doc.page_content)
        print('\n\n')

# RAG с ретривером
def rag_with_retrieval(question_data: dict, k: int = 5) -> str:
    """
    RAG пайплайн с поиском по FAISS.
    
    Args:
        question_data: словарь с данными вопроса
        k: количество чанков для извлечения
    """
    question = question_data['query']
    question_time = question_data['query_time']
    
    # Поиск релевантных чанков
    retrieved_docs = vectorstore.similarity_search(question, k=k)
    
    # Формируем контекст из найденных чанков
    context_parts = []
    for i, doc in enumerate(retrieved_docs):
        source = doc.metadata.get('source', 'unknown')
        context_parts.append(f"[Chunk {i+1} from {source}]\n{doc.page_content}")
    
    context = "\n\n".join(context_parts)
    
    # Вызываем LLM
    response = rag_chain.invoke({
        "context": context,
        "question": question,
        "question_time": question_time
    })
    
    return response, retrieved_docs


def test_rag(question_data: dict, k: int = 5, verbose: bool = True):
    """Тестирование RAG с выводом результатов."""
    response, retrieved_docs = rag_with_retrieval(question_data, k)
    
    print_chunks(retrieved_docs)
    print('\n------------------------')
    question_type = question_data.get('question_type')
    if question_type:
        print("Тип вопроса:", question_data.get('q'))
    print("Вопрос:", question_data['query'])
    answer = question_data.get('answer')
    if answer:
        print("Ожидаемый ответ:", question_data['answer'])
    print("\nОтвет RAG-системы:\n", response, sep='')
    
    return response

In [ ]:
i = 0
questions[i]

In [ ]:
_ = test_rag(questions[i], k=5, verbose=True)